In [1]:
import ray
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd


print(ray.__version__, pa.__version__)

2.2.0 10.0.1


In [2]:
ray.init(num_cpus=6)

2022-12-27 21:53:50,991	INFO worker.py:1535 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.7.15
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8265


In [3]:
from ray.data.datasource import RangeDatasource
source = RangeDatasource() 
ray.data.read_datasource(source, n=10).take() 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Table to Table Operation

1. can create(generate) new tables(paths) if not exist
2. can append new partitions
3. can re-run without side effect (idempotent)
4. check schema consistency before wrtie to tables


# About the Compute Engine

## Hive Partitioned IO

* Ray dataset --> modin dataset

# Create Toy Dataset

* pyarrow version 6.0.1 for python 3.6 - does not support overwrite
* [pyarrow support python 3.7 more](https://arrow.apache.org/docs/python/install.html)

In [4]:
!rm -r saved_by_pyarrow.parquet/*

In [5]:
d = [{'date':20220101,'value':1},{'date':20220102,'value':2}]

pdf = pd.DataFrame.from_records(d)

tb = pa.Table.from_pandas(pdf)


# The api
# https://arrow.apache .org/docs/python/generated/pyarrow.parquet.write_to_dataset.html
pq.write_to_dataset(
    tb,
    root_path='saved_by_pyarrow.parquet',
#     existing_data_behavior='overwrite_or_ignore',
    partition_cols=['date']
)


print(tb, type(tb))

pyarrow.Table
date: int64
value: int64
----
date: [[20220101,20220102]]
value: [[1,2]] <class 'pyarrow.lib.Table'>


In [6]:
!ls saved_by_pyarrow.parquet/*

saved_by_pyarrow.parquet/date=20220101:
c88b4f498f4b44c28aa81791275bab5f-0.parquet

saved_by_pyarrow.parquet/date=20220102:
c88b4f498f4b44c28aa81791275bab5f-0.parquet


## Read

In [7]:
print(dir(pq))

print(pq.ParquetDataset)

# pq.ParquetDataset?

['ColumnChunkMetaData', 'ColumnSchema', 'FileDecryptionProperties', 'FileEncryptionProperties', 'FileMetaData', 'ParquetDataset', 'ParquetDatasetPiece', 'ParquetFile', 'ParquetLogicalType', 'ParquetManifest', 'ParquetPartitions', 'ParquetReader', 'ParquetSchema', 'ParquetWriter', 'PartitionSet', 'RowGroupMetaData', 'Statistics', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_filters_to_expression', 'core', 'filters_to_expression', 'read_metadata', 'read_pandas', 'read_schema', 'read_table', 'write_metadata', 'write_table', 'write_to_dataset']
<class 'pyarrow.parquet.core.ParquetDataset'>


In [8]:
import logging

# monkey patching to earse thr argument

logging.warning(f"abc")

In [12]:
# from ray.data.datasource import ParquetDatasource

In [13]:
# ray.data.read_datasource( # doctest: +SKIP
#     source, paths="/path/to/dir").take()

In [10]:
# wrapper from ParquetDataset
# pds = ParquetDatasource(paths='saved_by_pyarrow.parquet')

pds = ParquetDatasource()

rd : ray.data.Dataset = ray.data.read_datasource(pds, paths='saved_by_pyarrow.parquet')
    
print(rd)

rd

(_get_read_tasks pid=29978) /Users/yulongtsai/opt/miniconda3/envs/ray_py37/lib/python3.7/site-packages/ray/data/datasource/parquet_datasource.py:233: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=29978)   pq_ds.pieces, **prefetch_remote_args
(_get_read_tasks pid=29978) /Users/yulongtsai/opt/miniconda3/envs/ray_py37/lib/python3.7/site-packages/ray/data/datasource/parquet_datasource.py:311: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=29978)   num_files = len(self._pq_ds.pieces)
(_get_read_tasks pid=29978) /Users/yulongtsai/opt/miniconda3/envs/ray_py37/lib/python3.7/site-packages/ray/data/datasource/parquet_datasource.py:325: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will

Dataset(num_blocks=2, num_rows=2, schema={value: int64, date: dictionary<values=int32, indices=int32, ordered=0>})


Parquet Files Sample: 100%|██████████| 2/2 [00:00<00:00, 4599.02it/s]
(_get_read_tasks pid=29978) /Users/yulongtsai/opt/miniconda3/envs/ray_py37/lib/python3.7/site-packages/ray/data/datasource/parquet_datasource.py:263: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=29978)   np.array_split(self._pq_ds.pieces, parallelism),


TypeError: _log() got an unexpected keyword argument 'stacklevel'

Dataset(num_blocks=2, num_rows=2, schema={value: int64, date: dictionary<values=int32, indices=int32, ordered=0>})

* `TypeError: _log() got an unexpected keyword argument 'stacklevel'` python 3.7 up support stacklevel in logging

* monkey patching

* downgrade ray 

TypeError: _log() got an unexpected keyword argument 'stacklevel'

Dataset(num_blocks=2, num_rows=2, schema={value: int64, date: dictionary<values=int32, indices=int32, ordered=0>})

## Write